<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 5 Assignment: Predicting Home Prices**

**Student Name: Your Name**

# Assignment Instructions

This assignment asks you to train a neural network to predict housing prices.  I provide you with two different datasets.  You will use one data set, which includes house prices for training.  A second dataset, which does not include house prices, will be used for prediction and be submitted for evaluation.  I also give you a third dataset that contains median income for zip codes that must be joined to both the training/test datasets to provide an additional input value.  You must use the median income with your inputs for extra predictive power.

You can find all of the needed CSV files here:

* [House Prices - Training](https://data.heatonresearch.com/data/t81-558/datasets/houses_train.csv)
* [House Prices - Submit](https://data.heatonresearch.com/data/t81-558/datasets/houses_test.csv)
* [Median Income by Zipcode](https://data.heatonresearch.com/data/t81-558/datasets/zips.csv)

The median income by zipcode provides an additional feature, median income, that you should use in your predictions.  To complete this assignment perform the following steps:

* Load the housing prices training data.
* Join the median income by zipcode to the training data so that you gain the median income.
* Train a model to predict house price when given the following inputs: 'bedrooms', 'bathrooms', 'garage', 'land', 'sqft', 'median_income'.
* Load the housing prices test data.  This data does not contain the house price, you must predict this.
* Join the median income by zipcode to the test/submit data to gain the median income.
* Predict prices for the test/submit data.  
* Create a submission dataset that contains the house id (from the test/submit data) and the predicted price for that house.  Include no other fields.
* Submit this dataset and see how close you are to the actual values.

Predicting the house prices with less than +/- $10,000 is sufficient to complete the assignment.  You may also wish to see if you can get your prediction even more accurate.

A few notes/suggestions on this assignment:

* You will see really high loss rates, due to the face that the target value is the price of a house (which is a large number)
* You might want to add a RMSE metric, which is the error in the same units target value. 
* I was able to obtain an RMSE of 86; yet still had a fairly high loss of 25920990.
* To add RMSE as a metric:
```
model.compile(loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer=opt)
```
# Google CoLab Instructions
If you are using Google CoLab, it will be necessary to mount your GDrive to send your notebook during the submit process. Running the following code will map your GDrive to /content/drive.

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems. 

**It is unlikely that should need to modify this function.**

In [ ]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #5 Sample Code

The following code provides a starting point for this assignment.

In [ ]:
import os
import pandas as pd
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scipy.stats import zscore

# This is your student key that I emailed to you at the beginnning of the semester.
key = "Gx5en9cEVvaZnjut6vfLm1HG4ZdddsI32sgldAXj"  # This is an example key and will not work.

# You must also identify your source file.  (modify for your local setup)
file='/content/drive/My Drive/Colab Notebooks/assignment_yourname_class5.ipynb'  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\assignments\\assignment_yourname_class5.ipynb'  # Windows
# file='/Users/jheaton/projects/t81_558_deep_learning/assignments/assignment_yourname_class5.ipynb'  # Mac/Linux

# Begin assignment
df_houses_train = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/houses_train.csv")
df_houses_submit = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/houses_test.csv")
df_zips = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/zips.csv")


submit(source_file=file,data=[df_submit],key=key,no=5)